In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')
sys.path.append('/home/rustam/hyperspecter_segmentation/')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter
import cv2
from hsi_dataset_api import HsiDataset
from makitorch.dataloaders.HsiDataloader import HsiDataloader

import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torchvision import utils
import cv2
from Losses import FocalLoss
import matplotlib.pyplot as plt

import seaborn as sns
import json

from my_super_attention_net_parts_diff_window_size_opt_change_lr_dyn_new_augs_v9 import *
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [ ]:
cut_window = (128, 128)

net = MySuperNetLittleInput(17, 17)
net.eval()
with torch.no_grad():
    _ = net(torch.randn(1, 17, cut_window[0], cut_window[1]))
net.train()

In [ ]:
import glob
w_sorted = sorted(glob.glob('pytorch_li_logs/262391742/*'), 
       key=lambda x: -float(x.split('/')[-1].split('-')[-1][9:13])
)

pick_best_one = w_sorted[1]
w_sorted

In [ ]:
model = NnModel.load_from_checkpoint(
    pick_best_one, 
    model=net, loss=nn.CrossEntropyLoss()
)

In [ ]:
net = model.model
net.eval()

In [ ]:
def test_augmentationv2(image, mask, **kwargs):
    image = torch.from_numpy(image)
    #image = (image - image.min()) / (image.max() - image.min())
    
    mask = torch.from_numpy(mask)
    mask = torch.squeeze(mask, 0)
    return image, mask

In [ ]:
test_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_test.npy')
# Test
dataset_creator_test = DatasetCreator(
    PATH_DATA, preprocessing=preprocessing, 
    indices=test_indices, cut_window=None,
    create_shared_memory=False
)
dataset_test = HsiDataloaderCutter(
    images=dataset_creator_test.images, masks=dataset_creator_test.masks,
    augmentation=test_augmentationv2,
    shuffle_data=False, cut_window=None,
)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1)

In [ ]:
device = 'cuda:0'

In [ ]:
net.to(device=device)

In [ ]:
def get_attention_pac_cam(net, x):
    x = net.backbone(x)
    _, attention_pam = net.pam_module(x, True)
    _, attention_cam = net.cam_module(x, True)
    return attention_pam, attention_cam

In [ ]:
data_iter = iter(test_loader)

In [ ]:
in_data_x, val_data = next(data_iter)

In [ ]:
cut_window=(128, 128); image_shape=(512, 512); num_classes=17; divided_batch=4;    
batch_size = in_data_x.shape[0]
# We will cut image into peases and stack it into single BIG batch
h_win, w_win = cut_window
h_parts, w_parts = image_shape[1] // w_win, image_shape[0] // h_win
in_data_x_parts_list = cut_into_parts_model_input(
    in_data_x, h_parts=h_parts, 
    w_parts=w_parts, h_win=h_win, w_win=w_win
)
target_parts_list = cut_into_parts_model_input(
    val_data.unsqueeze(dim=1), h_parts=h_parts, 
    w_parts=w_parts, h_win=h_win, w_win=w_win
)
in_data_x_batch = torch.cat(in_data_x_parts_list, dim=0) # (N, 17, 1, 1)
# Make predictions
part_divided = len(in_data_x_batch) // divided_batch
pred_batch_list = []
pred_attention_pam_batch_list = []
pred_attention_cam_batch_list = []

for b_i in range(part_divided):
    if b_i == (part_divided - 1):
        # last
        single_batch = in_data_x_batch[b_i * divided_batch:]
    else:
        single_batch = in_data_x_batch[b_i * divided_batch: (b_i+1) * divided_batch]
    # Make predictions
    with torch.no_grad():
        data = single_batch.to(device=device)
        preds = net(data).cpu().detach() # (divided_batch, num_classes)
        preds_attention_pam, preds_attention_cam  = get_attention_pac_cam(net, data)
        preds_attention_pam = preds_attention_pam.cpu().detach()
        preds_attention_cam = preds_attention_cam.cpu().detach()

    pred_batch_list.append(preds)
    pred_attention_pam_batch_list.append(preds_attention_pam)
    pred_attention_cam_batch_list.append(preds_attention_cam)

preds = torch.cat(pred_batch_list, dim=0)
pred_attention_pam = torch.cat(pred_attention_pam_batch_list, dim=0).view(16, 32, 32, 32, 32)
pred_attention_cam = torch.cat(pred_attention_cam_batch_list, dim=0).view(16, 128, 128)
target = torch.cat(target_parts_list, dim=0)
in_data_x_tensor = torch.cat(in_data_x_parts_list, dim=0)

In [ ]:
preds.shape, target.shape, pred_attention_pam.shape, pred_attention_cam.shape, in_data_x_tensor.shape

In [ ]:
pred_mask = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
pred_mask.shape

In [ ]:

h_win, w_win = cut_window
h_parts = 512 // h_win
w_parts = 512 // w_win


def merge_parts_into_single_mask(
        preds, shape, h_parts: int, 
        w_parts: int, h_win: int, w_win: int):
    pred_mask = torch.zeros(
        shape,
        dtype=preds.dtype, device=preds.device
    )
    counter = 0

    for h_i in range(h_parts):
        for w_i in range(w_parts):
            pred_mask[...,  
                h_i * h_win: (h_i+1) * h_win, 
                w_i * w_win: (w_i+1) * w_win
            ] = preds[counter]
            counter += 1
    return pred_mask

pred_big = merge_parts_into_single_mask(
    preds, shape=(1, 17, 512, 512), 
    h_parts=h_parts, w_parts=w_parts, h_win=h_win, w_win=w_win
)
pred_big_mask = torch.argmax(torch.nn.functional.softmax(pred_big, dim=1), dim=1)
target_big_mask = merge_parts_into_single_mask(
    target, shape=(1, 512, 512), 
    h_parts=h_parts, w_parts=w_parts, h_win=h_win, w_win=w_win
)

In [ ]:
fig = plt.figure(figsize=(12, 12))

fig.add_subplot(1, 2, 1)
sns.heatmap(pred_big_mask[0], vmin=0, vmax=17)

fig.add_subplot(1, 2, 2)
sns.heatmap(target_big_mask[0], vmin=0, vmax=17)


In [ ]:
indx_b = 15

In [ ]:
sns.heatmap(pred_mask[indx_b], vmax=17)

In [ ]:
sns.heatmap(target[indx_b][0], vmax=17)

In [ ]:
x, y = 2, 2
plt.plot(x,y,'bo') 
sns.heatmap(pred_attention_pam[indx_b][x, y])

In [ ]:
orig_size = 128
att_size = 32


attention_pam = torch.clone(pred_attention_pam[indx_b])

count_x = 4
count_y = 4

size_x = att_size // count_x
size_y = att_size // count_y
y_i, x_i = 0, 0
mid_x, mid_y = size_x // 2, size_y // 2

fig = plt.figure(figsize=(16, 16))

for i in range(1, count_x * count_y + 1):
    fig.add_subplot(count_x, count_y, i)
    if x_i == count_x:
        x_i = 0
        y_i += 1
    x, y = mid_x + x_i * size_x, mid_y + y_i * size_y
    att_s = attention_pam[y, x].unsqueeze(dim=0).unsqueeze(dim=0)
    att_s = torch.nn.functional.interpolate(att_s, (128, 128)).squeeze()
    sns.heatmap(att_s)
    
    x *= (orig_size // att_size)
    y *= (orig_size // att_size)
    plt.plot(x,y,'bo') 
    
    x_i += 1

In [ ]:
sns.heatmap(in_data_x_tensor[indx_b][0])

In [ ]:
fig = plt.figure(figsize=(16, 16))

for i in range(1, 17):
    fig.add_subplot(count_x, count_y, i)
    sns.heatmap(in_data_x_tensor[indx_b][i])

In [ ]:
s_data_x = in_data_x_tensor[indx_b]
in_data_x_tensor_normed = (s_data_x - s_data_x.min()) / (s_data_x.max() - s_data_x.min())

In [ ]:
sns.heatmap(in_data_x_tensor_normed[0])

In [ ]:
fig = plt.figure(figsize=(16, 16))

for i in range(1, 17):
    fig.add_subplot(count_x, count_y, i)
    sns.heatmap(in_data_x_tensor_normed[i])

In [ ]:
sns.heatmap(torch.sqrt(torch.sum(torch.square(in_data_x_tensor[indx_b]), dim=0)))

In [ ]:
sns.heatmap(torch.sum(in_data_x_tensor[indx_b], dim=0))

In [ ]:
plt.plot(x,y,'bo') 
sns.heatmap(pred_attention_cam[indx_b])

In [ ]:
sns.heatmap(torch.softmax(preds[indx_b, 5], dim=1))

In [ ]:
sns.heatmap(target_list[indx], vmax=17)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(
    np.asarray(target_list).reshape(-1),
    np.asarray(preds_list).reshape(-1), 
    average='macro'
)

In [ ]:
f1_score(
    np.asarray(target_list).reshape(-1),
    np.asarray(preds_list).reshape(-1), 
    average='weighted'
)

In [ ]:
f1_score(
    np.asarray(target_list).reshape(-1),
    np.asarray(preds_list).reshape(-1), 
    average=None
).mean()